In [1]:
#change working directory
import os
print(os.getcwd())
os.chdir('/home/student/A2/drive-download-20240507T073745Z-001')


from datetime import datetime, timedelta
import pymongo
from pymongo import MongoClient
#Residence wifi ip address: 172.16.62.212
ipadd = "10.192.33.112"


client = MongoClient(ipadd, 27017)
db = client.fit3182_db
collection = db.fit3182_assignment_db

#res = collection.find().sort('surface_temperature_celcius', pymongo.DESCENDING).limit(10)
res = collection.find().sort('date', pymongo.DESCENDING).limit(1)

for d in res:
    res = d
#     print(d)

latest_date = res['date']
#string to datetime object
# latest_date = datetime.strptime(latest_date, '%Y-%m-%d %H:%M:%S')
#string to datetime object
latest_date = datetime.strptime(latest_date, '%d/%m/%Y')
start_date = latest_date + timedelta(days=1)
print(start_date)

/home/student/A2
2024-01-01 00:00:00


In [2]:
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import random
import json
import pandas as pd

hostip = ipadd # change it to your IP

def connect_kafka_producer():
    producer = None
    
    try:
        producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],api_version=(0,10))
    
    except Exception as ex:
        print("Exception while connecting Kafka")
        print(str(ex))
    finally:
        return producer 
    
def publish_msg(producer, topic, data):
    try:
        #print(data)        #send the  value by dumps to JSON object
                            #then encode with utf-8 into bytes
        value_bytes = json.dumps(data).encode('utf-8')
        producer.send(topic, value=value_bytes)
        #print(data)
        """
        The flush() method in Kafka’s producer API is used to ensure that all 
        previously sent messages have been transmitted to the server and acknowledged 
        before proceeding. When you call flush(), it blocks the current thread and waits 
        for the producer to complete the sending of all records
        """
        producer.flush()  
    except Exception as ex:
        print('Exception from publish_msg func')
        print(str(ex))
    


In [ ]:
"""
Event Producer 2
"""
hotspot_AQUA = pd.read_csv("./hotspot_AQUA_streaming.csv", sep=',')

hotspot_AQUA.head()


"""
Main Function
"""
if __name__ == '__main__':
    topic = 'Producer2'
    print('Publishing records..')
    producer02 = connect_kafka_producer()
    
    
    rand_row_indices = list(range(len(hotspot_AQUA.index)))
    random.shuffle(rand_row_indices) 
    # random choose time from 1 to 10
    max_time = 3 #testing purpose
    start_time = start_date
    #rand_time = list(range(1,max_time))
#     random.shuffle(rand_time) 
     

    for index, hotspot_AQUA_row in hotspot_AQUA.iterrows():
        random_time = rand_row_indices[index]%max_time + 1
        
        hotspot_AQUA_row = hotspot_AQUA.loc[rand_row_indices[index]]

        hotspot_AQUA_doc = {
            'latitude': hotspot_AQUA_row['latitude'],
            'longitude': hotspot_AQUA_row['longitude'],
            'confidence': hotspot_AQUA_row['confidence'],
            'surface_temperature_celcius': hotspot_AQUA_row['surface_temperature_celcius'],
            'producer': "Producer_2",
            'created_time': start_time.strftime('%H:%M:%S')# at least 1 sec
        }
        #update start date
        #print(climate_doc['date'])
        #producer publish msg in strictured streaming
        # convert the dictionary to JSON-formatted string and encode as UFT-8 by publish_msg
        # msg = json.dumps(climate_doc)
        print(hotspot_AQUA_doc['created_time'])
        #print(hotspot_AQUA_doc['random_time'])
        publish_msg(producer02, topic, hotspot_AQUA_doc)
        #random minute for created_time
        start_time = start_time + timedelta(minutes=random_time)
        # every random seconds
        sleep(random_time) 



Publishing records..
00:00:00
00:01:00
00:03:00
00:05:00
00:08:00
00:11:00
00:12:00
00:15:00
00:18:00
00:20:00
00:21:00
00:24:00
00:27:00
00:28:00
00:31:00
00:32:00
00:34:00
00:36:00
00:38:00
00:40:00
00:42:00
00:44:00
00:47:00
00:48:00
00:51:00
00:52:00
00:55:00
00:56:00
00:57:00
01:00:00
01:02:00
01:05:00
01:07:00
01:09:00
01:10:00
01:12:00
01:14:00
01:15:00
01:18:00
01:21:00
01:24:00
01:25:00
01:28:00
01:29:00
01:30:00
01:32:00
01:35:00
01:38:00
01:41:00
01:42:00
01:44:00
01:45:00
01:47:00
01:50:00
01:51:00
01:54:00
01:56:00
01:58:00
01:59:00
02:02:00
02:04:00
02:05:00
02:08:00
02:11:00
02:14:00
02:17:00
02:18:00
02:20:00
02:21:00
02:22:00
02:24:00
02:25:00
02:28:00
02:30:00
02:33:00
02:34:00
02:36:00
02:39:00
02:42:00
02:43:00
02:44:00
02:45:00
02:47:00
02:50:00
02:53:00
02:56:00
02:59:00
03:02:00
03:03:00
03:06:00
03:07:00
03:10:00
03:13:00
03:15:00
03:16:00
03:19:00
03:20:00
03:22:00
03:24:00
03:25:00
03:26:00
03:28:00
03:30:00
03:31:00
03:33:00
03:35:00
03:37:00
03:38:00
03:41:0

05:52:00
05:54:00
05:57:00
05:58:00
06:01:00
06:02:00
06:04:00
06:07:00
06:08:00
06:10:00
06:13:00
06:16:00
06:19:00
06:21:00
06:23:00
06:25:00
06:28:00
06:30:00
06:31:00
06:32:00
06:34:00
06:36:00
06:39:00
06:40:00
06:41:00
06:43:00
06:46:00
06:49:00
06:52:00
06:54:00
06:56:00
06:57:00
07:00:00
07:02:00
07:04:00
07:06:00
07:07:00
07:09:00
07:12:00
07:14:00
07:16:00
07:19:00
07:21:00
07:23:00
